In [1]:
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

In [2]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.16-py3-none-any.whl size=110685 sha256=80e8344ee439733c056110b45a27c043f8a582d90fc33bea6c1fed8dbd47ff54
  Stored in directory: /root/.cache/pip/wheels/d2/ed/a5/da3a0cfb13373d1ace41cafa4f2467d858c55c52473ba72799
Successfully built kaggle


In [3]:
# !cp ../kaggle.json /root/.kaggle/kaggle.json


In [4]:
# Copying over from JH notebook
import os
from pathlib import Path
import pandas as pd
import numpy as np

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    path = Path('titanic')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

In [5]:
import pandas as pd
import numpy as np
import copy
orig_train_df = pd.read_csv(path/'train.csv')
train_df = copy.deepcopy(orig_train_df)

In [6]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [7]:
# TODO: Check which ones JH included
cat_vars = ['Pclass', 'Sex', 'Embarked']
num_vars = ['Age', 'SibSp', 'Parch', 'Fare']

In [8]:
num_df = train_df[num_vars].fillna(train_df[num_vars].mode().iloc[0])
num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)

# Going through an easier approach rather than typical norm
num_df = num_df / num_df.max()
num_df

,Age,SibSp,Parch,Fare
0,0.2750,0.125,0.000000,0.317521
1,0.4750,0.125,0.000000,0.683873
2,0.3250,0.000,0.000000,0.331789
3,0.4375,0.125,0.000000,0.636672
4,0.4375,0.000,0.000000,0.334298
...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118
887,0.2375,0.000,0.000000,0.545154
888,0.3000,0.125,0.333333,0.505672
889,0.3250,0.000,0.000000,0.545154


In [9]:
train_df[cat_vars].describe(include='all')
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
train_df.Pclass.unique()

array([3, 1, 2])

In [11]:
# Curious how in-depth you should conside N/As
cat_df = pd.get_dummies(train_df[cat_vars], columns=cat_vars)
cat_df

,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1,0,1,0,0,1
1,1,0,0,1,0,1,0,0
2,0,0,1,1,0,0,0,1
3,1,0,0,1,0,0,0,1
4,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...
886,0,1,0,0,1,0,0,1
887,1,0,0,1,0,0,0,1
888,0,0,1,1,0,0,0,1
889,1,0,0,0,1,1,0,0


In [12]:
# Might need to normalize after creating the dummy vars?
proc_train_df = num_df.join(cat_df)

In [13]:
# Realize, going back to this, that we're not saving the mode but taking it for each piece of data we bring in
def process_df(path, has_dependent=True):
    df = pd.read_csv(path)
    df = df.fillna(df.mode().iloc[0])
    
    
    # TODO: Check which ones JH included
    cat_vars = ['Pclass', 'Sex', 'Embarked']
    num_vars = ['Age', 'SibSp', 'Parch', 'Fare']
    
    
    num_df = df[num_vars]
    num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)
    # Going for an easier approach rather than typical norm
    num_df = num_df / num_df.max()

    cat_df = pd.get_dummies(df[cat_vars], columns=cat_vars)
    cat_df['const'] = 1
    
    if has_dependent:
        return (num_df.join(cat_df), df['Survived'])
    else:
        return num_df.join(cat_df)

In [14]:
train_df, train_dep = process_df(path/'train.csv')
t_dep = torch.tensor(train_dep)
train_df

/tmp/ipykernel_84/3771005558.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0,1
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0,1


In [15]:
train_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [16]:
test_df = process_df(path/'test.csv', has_dependent=False)
test_df

/tmp/ipykernel_84/3771005558.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_df['Fare'] = np.log(num_df['Fare'] + 1e-10)


,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.453947,0.000,0.000000,0.329840,0,0,1,0,1,0,1,0,1
1,0.618421,0.125,0.000000,0.311896,0,0,1,1,0,0,0,1,1
2,0.815789,0.000,0.000000,0.363976,0,1,0,0,1,0,1,0,1
3,0.355263,0.000,0.000000,0.346051,0,0,1,0,1,0,0,1,1
4,0.289474,0.125,0.111111,0.402083,0,0,1,1,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
414,0.513158,0.000,0.000000,0.751796,1,0,0,1,0,1,0,0,1
415,0.506579,0.000,0.000000,0.317521,0,0,1,0,1,0,0,1,1
416,0.276316,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1


In [17]:
tst_indep = torch.tensor(test_df.values)
tst_indep

tensor([[0.4539, 0.0000, 0.0000, 0.3298, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.6184, 0.1250, 0.0000, 0.3119, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.8158, 0.0000, 0.0000, 0.3640, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.3553, 0.0000, 0.0000, 0.3461, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.2895, 0.1250, 0.1111, 0.4021, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.1842, 0.0000, 0.0000, 0.3561, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.3947, 0.0000, 0.0000, 0.3257, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        ...,
        [0.4868, 0.1250, 0.0000, 0.7212, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.3684, 0.0000, 0.0000, 0.3287, 0.0000, 0.0000, 1.0000, 1.0000, 0.

In [18]:
test_df.isna().sum()

Age           0
SibSp         0
Parch         0
Fare          0
Pclass_1      0
Pclass_2      0
Pclass_3      0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
const         0
dtype: int64

In [19]:
# Great success! We can move into building out model!

In [20]:
import torch

In [21]:
weights = torch.rand(train_df.shape[1]) - 0.5
weights

tensor([-0.0283,  0.4963,  0.4554, -0.4240, -0.1928,  0.0285, -0.1892, -0.2169, -0.4901,  0.0793, -0.1776,  0.1938,  0.4816])

In [22]:
tensor_df = torch.tensor(train_df.values)
tensor_df

tensor([[0.2750, 0.1250, 0.0000, 0.3175, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4750, 0.1250, 0.0000, 0.6839, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3250, 0.0000, 0.0000, 0.3318, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.1250, 0.0000, 0.6367, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.0000, 0.0000, 0.3343, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.3000, 0.0000, 0.0000, 0.3422, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.6750, 0.0000, 0.0000, 0.6329, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        ...,
        [0.3125, 0.0000, 0.0000, 0.3130, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4875, 0.0000, 0.8333, 0.5404, 0.0000, 0.0000, 1.0000, 1.0000, 0.

In [23]:
result = (tensor_df * weights)
result

tensor([[-0.0078,  0.0620,  0.0000, -0.1346, -0.0000,  0.0000, -0.1892, -0.0000, -0.4901,  0.0000, -0.0000,  0.1938,  0.4816],
        [-0.0134,  0.0620,  0.0000, -0.2900, -0.1928,  0.0000, -0.0000, -0.2169, -0.0000,  0.0793, -0.0000,  0.0000,  0.4816],
        [-0.0092,  0.0000,  0.0000, -0.1407, -0.0000,  0.0000, -0.1892, -0.2169, -0.0000,  0.0000, -0.0000,  0.1938,  0.4816],
        [-0.0124,  0.0620,  0.0000, -0.2700, -0.1928,  0.0000, -0.0000, -0.2169, -0.0000,  0.0000, -0.0000,  0.1938,  0.4816],
        [-0.0124,  0.0000,  0.0000, -0.1418, -0.0000,  0.0000, -0.1892, -0.0000, -0.4901,  0.0000, -0.0000,  0.1938,  0.4816],
        [-0.0085,  0.0000,  0.0000, -0.1451, -0.0000,  0.0000, -0.1892, -0.0000, -0.4901,  0.0000, -0.1776,  0.0000,  0.4816],
        [-0.0191,  0.0000,  0.0000, -0.2684, -0.1928,  0.0000, -0.0000, -0.0000, -0.4901,  0.0000, -0.0000,  0.1938,  0.4816],
        ...,
        [-0.0088,  0.0000,  0.0000, -0.1327, -0.0000,  0.0000, -0.1892, -0.0000, -0.4901,  0.0000,

In [24]:
result.shape

torch.Size([891, 13])

In [25]:
train_df.shape

(891, 13)

In [26]:
def sigmoid(x):
    return 1 / (1 + 1 / torch.exp(-x))

In [27]:
torch_implementation_sigmoid = torch.sigmoid(result.sum(axis=1))
my_implementation_sigmoid = sigmoid(result.sum(axis=1))

In [28]:
# Awesome!
(torch_implementation_sigmoid == torch_implementation_sigmoid).sum()

tensor(891)

In [29]:
weights

tensor([-0.0283,  0.4963,  0.4554, -0.4240, -0.1928,  0.0285, -0.1892, -0.2169, -0.4901,  0.0793, -0.1776,  0.1938,  0.4816])

In [30]:
# TODO: Implement actual training

import torch
torch.manual_seed(0)
import random
random.seed(0)
import numpy as np
np.random.seed(0)

def mae(preds, actual):
    return torch.abs(preds - actual).mean()

actual = torch.tensor(orig_train_df['Survived'])
weights = torch.rand(train_df.shape[1]) - 0.5
weights.requires_grad_()

def training_step(step, lr):
    global weights
    
    layer_results = tensor_df * weights
    preds = sigmoid(layer_results.sum(axis=1))
    loss = mae(preds, actual)

    
    if step % 100 == 0:
        print("loss:", loss)

    # print(weights.data)
        
    # if torch.isnan(loss):
    #     print(weights)
        
    loss.backward()
    
    with torch.no_grad():
        weights -= lr * weights.grad
        
def train(steps=10000, lr=3e-3):
    for i in range(steps):
        training_step(i, lr)
        
def pred(input_tensor):
    layer_results = input_tensor * weights
    return sigmoid(layer_results.sum(axis=1))

train()

loss: tensor(0.5500, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.3298, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2631, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2053, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2049, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2095, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2113, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2120, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2123, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2125, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2126, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2126, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2125, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2124, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss: tensor(0.2122, dtype=torch.float64, grad_fn=<MeanBackwar

In [31]:
weights.data

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [32]:
weights - weights.grad

tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], grad_fn=<SubBackward0>)

In [33]:
# Since weights are blowing up, going back through and checking my notebook against JH
# TODO: Plug in fastbook to write these up; learn in public bb
train_df

,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,const
0,0.2750,0.125,0.000000,0.317521,0,0,1,0,1,0,0,1,1
1,0.4750,0.125,0.000000,0.683873,1,0,0,1,0,1,0,0,1
2,0.3250,0.000,0.000000,0.331789,0,0,1,1,0,0,0,1,1
3,0.4375,0.125,0.000000,0.636672,1,0,0,1,0,0,0,1,1
4,0.4375,0.000,0.000000,0.334298,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.3375,0.000,0.000000,0.411118,0,1,0,0,1,0,0,1,1
887,0.2375,0.000,0.000000,0.545154,1,0,0,1,0,0,0,1,1
888,0.3000,0.125,0.333333,0.505672,0,0,1,1,0,0,0,1,1
889,0.3250,0.000,0.000000,0.545154,1,0,0,0,1,1,0,0,1


## Exploring NaN weights

- Like the options to change display:

```python
import torch, numpy as np, pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)
```

- Doesn't look like he "normalized" numerical values
- cat_vars = [sex, pclass, embarked] which lines up with mine!
- num_vars line up as well, Age, SibSp, Parch, and Fare.
- Primary difference looks like he didn't
- Also his naming scheme looks a lot cleaner than mine; something to learn here
- I added a const column, might be better, but JH might've been just going the easy way

- Ahhhhh looks like problem might be `coeffs.grad.zero_()`

In [34]:
weights = torch.rand(train_df.shape[1]) - 0.5
tensor_df * weights

tensor([[-0.0911, -0.0258,  0.0000,  0.0628,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.1573, -0.0258,  0.0000,  0.1352,  0.3000, -0.0000, -0.0000,  0.1816,  0.0000, -0.1029,  0.0000, -0.0000,  0.0529],
        [-0.1076, -0.0000,  0.0000,  0.0656,  0.0000, -0.0000, -0.2177,  0.1816,  0.0000, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.1449, -0.0258,  0.0000,  0.1258,  0.3000, -0.0000, -0.0000,  0.1816,  0.0000, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.1449, -0.0000,  0.0000,  0.0661,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,  0.0000, -0.0806,  0.0529],
        [-0.0993, -0.0000,  0.0000,  0.0676,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,  0.3742, -0.0000,  0.0529],
        [-0.2235, -0.0000,  0.0000,  0.1251,  0.3000, -0.0000, -0.0000,  0.0000,  0.4152, -0.0000,  0.0000, -0.0806,  0.0529],
        ...,
        [-0.1035, -0.0000,  0.0000,  0.0619,  0.0000, -0.0000, -0.2177,  0.0000,  0.4152, -0.0000,

In [35]:
tensor_df.shape, weights.shape

(torch.Size([891, 13]), torch.Size([13]))

In [36]:
tensor_df

tensor([[0.2750, 0.1250, 0.0000, 0.3175, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4750, 0.1250, 0.0000, 0.6839, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000],
        [0.3250, 0.0000, 0.0000, 0.3318, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.1250, 0.0000, 0.6367, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4375, 0.0000, 0.0000, 0.3343, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.3000, 0.0000, 0.0000, 0.3422, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 1.0000],
        [0.6750, 0.0000, 0.0000, 0.6329, 1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        ...,
        [0.3125, 0.0000, 0.0000, 0.3130, 0.0000, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 1.0000],
        [0.4875, 0.0000, 0.8333, 0.5404, 0.0000, 0.0000, 1.0000, 1.0000, 0.

In [37]:
weights.shape

torch.Size([13])

In [38]:
!git config --global user.email "treyroyalt@gmail.com"
!git config --global user.name "jpr2000"
!git commit -m "Fix training with validation stats"

On branch lesson_5_from_scratch
Your branch is up to date with 'origin/lesson_5_from_scratch'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   nn_from_scrath_replication.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	titanic/

no changes added to commit (use "git add" and/or "git commit -a")


In [39]:
preds = (tensor_df*weights).sum(axis=1)
preds[:10]

tensor([ 0.1157,  0.3838, -0.1058,  0.4091,  0.0910,  0.5928,  0.5891,  0.1839, -0.0930, -0.1832], dtype=torch.float64)

In [40]:
loss = torch.abs(t_dep - preds).mean()
loss

tensor(0.4450, dtype=torch.float64)

In [41]:
def calc_preds(inputs, weights): return sigmoid((inputs*weights).sum(axis=1))
def calc_loss(inputs, weights, actual): 
    return torch.abs(actual - calc_preds(inputs, weights)).mean()

In [42]:
calc_loss(tensor_df, weights, t_dep)

tensor(0.4949, dtype=torch.float64)

In [43]:
weights

tensor([-0.3311, -0.2061,  0.0185,  0.1977,  0.3000, -0.3390, -0.2177,  0.1816,  0.4152, -0.1029,  0.3742, -0.0806,  0.0529])

In [44]:
# Going to see where we went wrong

import torch
import random
import numpy as np

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

weights = torch.rand(train_df.shape[1]) - 0.5
weights.requires_grad_()

def training_step(step, lr):
    global weights
    
    # layer_results = tensor_df * weights
    # preds = sigmoid(layer_results.sum(axis=1))
    # loss = mae(preds, actual)
    
    loss = calc_loss(tensor_df, weights, t_dep)

    # print(weights.data)
        
    # if torch.isnan(loss):
    #     print(weights)
        
    loss.backward()
    
    with torch.no_grad():
        weights.sub_(weights.grad * lr)
        
        # Think the following line is where we went wrong
        weights.grad.zero_()
        if step % 100 == 0:
            print("loss:", loss)
            print("val loss:", )
        
        
def train(steps=10000, lr=0.1):
    global weights
    weights = torch.rand(train_df.shape[1]) - 0.5
    weights.requires_grad_()
    
    for i in range(steps):
        training_step(i, lr)
        
train()

loss: tensor(0.4949, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.3642, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.3171, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2867, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2683, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2567, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2488, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2431, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2388, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2354, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2327, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2304, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.2286, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:

Results now look similar to Jeremy's! Importance of zeroing out your gradients!

In [45]:
weights

tensor([ 0.2976,  1.1950,  1.0525, -0.3461, -1.6866, -1.7561,  3.2334, -4.5529,  5.1971, -0.7122, -1.3137,  2.2640,  0.1003],
       requires_grad=True)

In [46]:
train(lr=1e2)

loss: tensor(0.4976, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1892, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1889, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1888, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1888, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1887, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1887, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1887, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1887, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1887, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1887, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1886, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:
loss: tensor(0.1886, dtype=torch.float64, grad_fn=<MeanBackward0>)
val loss:

## Let's actually build out a validation set

In [47]:
from fastai.data.transforms import RandomSplitter

train_idx, val_idx = RandomSplitter(seed=42)(train_df)

trn_indep, val_indep = tensor_df[train_idx], tensor_df[val_idx]
trn_dep, val_dep     = t_dep[train_idx], t_dep[val_idx]

In [48]:
# Verifying we still have all our data 
assert trn_indep.shape[0] + val_indep.shape[0] == 891

In [49]:
??calc_loss

Signature: calc_loss(inputs, weights, actual)
Docstring: <no docstring>
Source:   
def calc_loss(inputs, weights, actual): 
    return torch.abs(actual - calc_preds(inputs, weights)).mean()
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


In [50]:
# Updating for accuracy
# TODO: Need a better naming scheme

# TODO: Update this with new variables

import torch
import random
import numpy as np

weights

def training_step(step, lr):
    global weights
    
    loss = calc_loss(trn_indep, weights, trn_dep)
    loss.backward()
    
    with torch.no_grad():
        weights.sub_(weights.grad * lr)
        weights.grad.zero_()
        
        if step % 1000 == 0:
            trn_loss = calc_loss(trn_indep, weights, trn_dep).item()
            val_loss = calc_loss(val_indep, weights, val_dep).item()
            print(trn_loss, val_loss)
        
        
def train(steps=10000, lr=0.1):
    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)
    
    global weights
    weights = torch.rand(train_df.shape[1]) - 0.5
    weights.requires_grad_()
    
    for i in range(steps):
        training_step(i, lr)

In [51]:
train(lr=1e0)

0.5303622207049395 0.5319113867953892
0.20148153446224695 0.18740216158440876
0.19735286147191075 0.18160683233195013
0.19579616599382862 0.17934296847835493
0.1949595229515259 0.17811480634403976
0.19443323892736752 0.1773427211462587
0.19407070990278433 0.1768129781311443
0.19380553766900835 0.17642739246372777
0.1936030634730265 0.17613443709475138
0.19344336782294858 0.17590446493743944


In [52]:
??calc_loss

Signature: calc_loss(inputs, weights, actual)
Docstring: <no docstring>
Source:   
def calc_loss(inputs, weights, actual): 
    return torch.abs(actual - calc_preds(inputs, weights)).mean()
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


In [54]:
# Verifying we still have all our data 
trn_indep.shape[0] + val_indep.shape[0]

891

In [62]:
??calc_preds

Signature: calc_preds(inputs, weights)
Docstring: <no docstring>
Source:    def calc_preds(inputs, weights): return sigmoid((inputs*weights).sum(axis=1))
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


In [65]:
tst_indep.shape

torch.Size([418, 13])

In [77]:
preds = calc_preds(tst_indep, weights)
preds

tensor([    0.0000,     0.0245,     0.0202,     0.0000,     0.0449,     0.0000,     0.9900,     0.0000,     0.9790,     0.0000,     0.0000,
            0.0000,     0.9978,     0.0000,     0.9955,     1.0000,     0.0446,     0.0000,     0.0438,     0.9545,     0.0049,     0.0000,
            0.9985,     0.0169,     1.0000,     0.0000,     1.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.0293,
            0.0417,     0.0103,     0.0000,     0.0748,     0.0752,     0.0000,     0.0000,     0.0000,     0.0000,     0.0000,     0.9984,
            0.9957,     0.0000,     0.0092,     0.0000,     1.0000,     0.0385,     0.0000,     0.0196,     0.9971,     0.9931,     0.0234,
            0.0000,     0.0000,     0.0000,     0.0000,     1.0000,     0.0000,     0.0000,     0.0000,     0.9921,     0.0095,     0.9988,
            0.9930,     0.0000,     0.0139,     0.9890,     0.9916,     0.0000,     0.0599,     0.0149,     1.0000,     0.0154,     0.0000,
            0.9910, 

In [85]:
(preds > 0.5).sum() / preds.shape[0]

tensor(0.2656)

In [90]:
# CSV submission time!
# Found on stack overflow https://stackoverflow.com/questions/36158058/torch-save-tensor-to-csv-file
preds_np = preds.detach().numpy() #convert to Numpy array
preds_df = pd.DataFrame(t_np) #convert to a dataframe
# df.to_csv("testfile",index=False) #save to file


In [94]:
preds_df

,0
0,4.660216e-06
1,2.446008e-02
2,2.016978e-02
3,3.662470e-09
4,4.488586e-02
...,...
413,4.350830e-09
414,9.999956e-01
415,2.569265e-09
416,4.350830e-09


In [92]:
!pwd

/notebooks/fastai_work/replication


In [93]:
test_df = pd.read_csv('titanic/test.csv')
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [100]:
preds_df['PassengerId'] = test_df['PassengerId']
preds_df.rename(columns={0: 'Survived'}, inplace=True)
preds_df = preds_df[['PassengerId', 'Survived']]
preds_df

,PassengerId,Survived
0,892,4.660216e-06
1,893,2.446008e-02
2,894,2.016978e-02
3,895,3.662470e-09
4,896,4.488586e-02
...,...,...
413,1305,4.350830e-09
414,1306,9.999956e-01
415,1307,2.569265e-09
416,1308,4.350830e-09


In [103]:
preds_df['Survived'] = preds_df['Survived'].apply(lambda pred: 1 if pred >= 0.5 else 0)
preds_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [106]:
preds_df.to_csv(path/'submissions.csv', index=False)

## Trying to build DNN

In [137]:
trn_indep = trn_indep.float()

In [144]:
weights.shape

torch.Size([13])

In [138]:
# 713x13 @ 13x1 = 713x1
(trn_indep@weights).shape

torch.Size([713])

In [150]:
# So for 3 layers...
weights_3 = torch.rand(trn_indep.shape[1], 3) - 0.5
print(weights_3.shape)
weights_3

torch.Size([13, 3])


tensor([[ 0.3532, -0.2323,  0.4296],
        [-0.0981, -0.1847,  0.2546],
        [-0.0486, -0.3390,  0.0537],
        [-0.2328, -0.3618, -0.3155],
        [ 0.1735,  0.0421, -0.2026],
        [ 0.1785, -0.1565, -0.3859],
        [ 0.1717, -0.1703, -0.1482],
        [ 0.3594,  0.3737, -0.4894],
        [ 0.4288,  0.4471, -0.3606],
        [-0.4985, -0.3903, -0.1327],
        [-0.0225,  0.1378,  0.1193],
        [ 0.0969, -0.3903,  0.4812],
        [ 0.4307,  0.2436,  0.1092]])

In [153]:
??calc_preds

Signature: calc_preds(inputs, weights)
Docstring: <no docstring>
Source:    def calc_preds(inputs, weights): return sigmoid((inputs*weights).sum(axis=1))
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


In [158]:
(trn_indep*weights).shape, trn_indep*weights

(torch.Size([713, 13]),
 tensor([[ 0.0282,  0.3899,  0.4677, -0.2318, -0.0000, -0.0000,  6.0422, -0.0000,  8.5909, -0.0000, -0.0000,  4.5434, -0.3862],
         [ 1.1405,  0.0000,  0.0000, -0.1569, -0.0000, -0.0000,  6.0422, -0.0000,  8.5909, -0.0000, -2.8353,  0.0000, -0.3862],
         [ 0.7603,  0.0000,  0.0000, -0.1655, -0.0000, -0.0000,  6.0422, -0.0000,  8.5909, -0.0000, -0.0000,  4.5434, -0.3862],
         [ 0.8448,  0.3899,  0.0000, -0.2130, -0.0000, -0.0000,  6.0422, -0.0000,  8.5909, -0.0000, -0.0000,  4.5434, -0.3862],
         [ 0.0845,  1.1696,  0.2339, -0.2336, -0.0000, -0.0000,  6.0422, -8.1473,  0.0000, -0.0000, -0.0000,  4.5434, -0.3862],
         [ 0.9574,  0.0000,  0.2339, -0.2403, -0.0000, -3.1499,  0.0000, -8.1473,  0.0000, -0.0000, -0.0000,  4.5434, -0.3862],
         [ 0.8729,  0.3899,  0.0000, -0.3625, -2.8693, -0.0000,  0.0000, -8.1473,  0.0000, -1.7337, -0.0000,  0.0000, -0.3862],
         ...,
         [ 0.5069,  0.0000,  0.0000, -0.1572, -0.0000, -0.0000,  6

In [159]:
(trn_indep@weights).shape, trn_indep@weights

(torch.Size([713]),
 tensor([ 19.4443,  12.3950,  19.3851,  19.8119,   3.3064,  -6.1890, -12.2362,  19.4204,  19.4208,  10.3775,  10.6138,  10.4036,  19.3077,
          20.4986,  -6.3228,  -4.8078,  -6.3628,   2.5681,  11.3683,  10.5714,  10.8406, -11.5391,  19.3203,  19.3143,  19.5334,  19.8694,
           2.5411,  19.5624,  10.6399,  -5.0014,  19.1373,   2.9611,  -5.9265,  -6.1252,  19.3112,   3.9425,  19.1948,  10.0207,  21.0899,
           4.0890,  -6.3963,  -6.3564,   2.4498,   3.4472,  -6.2945,  -6.4530,  19.5436,  20.4532,   4.3878,  19.9619,  20.1117,  12.3192,
           2.6537,  19.1832,  19.4752,  19.5541,  10.6420,  19.0810,  19.3077,  -6.0338,   9.9179,  19.2385,  -4.4710,   3.8616,  10.4998,
           3.5403,  -6.1689,  -5.0331,  10.3937,   2.6256,  -4.8090, -11.7620,  10.7533,  19.3063,   4.4421,  10.8444, -12.5549,   4.4735,
          10.9792,   3.3437,  10.9182,  10.7122,   2.5727,  -6.5036,  -3.5567, -12.5574,  11.2590,  10.6847,  19.3383,  11.2149,  21.9347,
       

In [164]:
weights_3

tensor([[ 0.3532, -0.2323,  0.4296],
        [-0.0981, -0.1847,  0.2546],
        [-0.0486, -0.3390,  0.0537],
        [-0.2328, -0.3618, -0.3155],
        [ 0.1735,  0.0421, -0.2026],
        [ 0.1785, -0.1565, -0.3859],
        [ 0.1717, -0.1703, -0.1482],
        [ 0.3594,  0.3737, -0.4894],
        [ 0.4288,  0.4471, -0.3606],
        [-0.4985, -0.3903, -0.1327],
        [-0.0225,  0.1378,  0.1193],
        [ 0.0969, -0.3903,  0.4812],
        [ 0.4307,  0.2436,  0.1092]])

In [166]:
weights_1 = torch.rand(weights_3.shape[1], 1)
weights_1

tensor([[0.2208],
        [0.5365],
        [0.1432]])

In [163]:
# 713x13@13x3 = 713x3 = Nxn_dim
trn_indep@weights_3

tensor([[ 0.9912, -0.1842, -0.0161],
        [ 1.1112,  0.4219, -0.1663],
        [ 1.1667, -0.0735,  0.1176],
        [ 1.1446, -0.1412,  0.1341],
        [ 0.9133, -0.2545, -0.0807],
        [ 1.0904, -0.2666, -0.2519],
        [ 0.4132, -0.1183, -0.7565],
        ...,
        [ 1.0616, -0.1145, -0.0542],
        [ 2.1463,  1.3610,  1.4394],
        [ 1.1436, -0.0509,  0.0898],
        [ 1.2012, -0.0057,  0.0950],
        [ 0.9683,  0.3955, -0.3846],
        [ 0.5136, -0.1135, -0.5002],
        [ 1.1981, -0.1094, -0.0924]])

In [170]:
preds_no_activ = trn_indep@weights_3@weights_1
preds_no_activ.shape, preds_no_activ[:10]

(torch.Size([713, 1]),
 tensor([[ 0.1177],
         [ 0.4479],
         [ 0.2350],
         [ 0.1962],
         [ 0.0535],
         [ 0.0617],
         [-0.0806],
         [ 0.2395],
         [ 0.2398],
         [ 0.1312]]))

In [171]:
??calc_loss

Signature: calc_loss(inputs, weights, actual)
Docstring: <no docstring>
Source:   
def calc_loss(inputs, weights, actual): 
    return torch.abs(actual - calc_preds(inputs, weights)).mean()
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


In [172]:
??calc_preds

Signature: calc_preds(inputs, weights)
Docstring: <no docstring>
Source:    def calc_preds(inputs, weights): return sigmoid((inputs*weights).sum(axis=1))
File:      /tmp/ipykernel_84/3909039809.py
Type:      function


Signature: torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor
Source:   
def relu(input: Tensor, inplace: bool = False) -> Tensor:
    r"""relu(input, inplace=False) -> Tensor

    Applies the rectified linear unit function element-wise. See
    :class:`~torch.nn.ReLU` for more details.
    """
    if has_torch_function_unary(input):
        return handle_torch_function(relu, (input,), input, inplace=inplace)
    if inplace:
        result = torch.relu_(input)
    else:
        result = torch.relu(input)
    return result
File:      /usr/local/lib/python3.9/dist-packages/torch/nn/functional.py
Type:      function


In [237]:
import math
def relu(x):
    return math.max(0, x)

In [242]:
# Each layer gets its own matrix. Each matrix is its own transform like in graphics
# old_dim x new_dim
# prev.shape[1] x n_out

# Need the last layer to give us one dimension prediction

# TODO: Why relu -> sigmoid results in 0. But sigmoids all the way down results in 0.3?
units_per_layer = [5,3,1]

torch.manual_seed(0)

layer_1 = torch.rand(trn_indep.shape[1], units_per_layer[0]).requires_grad_()
layer_2 = torch.rand(units_per_layer[0], units_per_layer[1]).requires_grad_()
layer_3 = torch.rand(units_per_layer[1], units_per_layer[2]).requires_grad_()

layer_1_out = sigmoid(trn_indep@layer_1)
layer_2_out = sigmoid(layer_1_out@layer_2)
layer_3_out = sigmoid(layer_2_out@layer_3)
layer_3_out

tensor([[0.3260],
        [0.3130],
        [0.3308],
        [0.3258],
        [0.3124],
        [0.3158],
        [0.2991],
        [0.3309],
        [0.3309],
        [0.3106],
        [0.3152],
        [0.3300],
        [0.3314],
        [0.3171],
        [0.3178],
        [0.3064],
        [0.3186],
        [0.3206],
        [0.3103],
        [0.3258],
        [0.3280],
        [0.2985],
        [0.3313],
        [0.3316],
        [0.3305],
        [0.3291],
        [0.3207],
        [0.3304],
        [0.3208],
        [0.2991],
        [0.3320],
        [0.3161],
        [0.3113],
        [0.3054],
        [0.3315],
        [0.3066],
        [0.3318],
        [0.3278],
        [0.4266],
        [0.3043],
        [0.3148],
        [0.3059],
        [0.3208],
        [0.3119],
        [0.3127],
        [0.3045],
        [0.3295],
        [0.3241],
        [0.3054],
        [0.3291],
        [0.3238],
        [0.4135],
        [0.3203],
        [0.3314],
        [0.3306],
        [0

In [212]:
print(layer_1, '\n')
print(layer_2, '\n')
print(layer_3, '\n')

tensor([[0.4963, 0.7682, 0.0885, 0.1320, 0.3074],
        [0.6341, 0.4901, 0.8964, 0.4556, 0.6323],
        [0.3489, 0.4017, 0.0223, 0.1689, 0.2939],
        [0.5185, 0.6977, 0.8000, 0.1610, 0.2823],
        [0.6816, 0.9152, 0.3971, 0.8742, 0.4194],
        [0.5529, 0.9527, 0.0362, 0.1852, 0.3734],
        [0.3051, 0.9320, 0.1759, 0.2698, 0.1507],
        [0.0317, 0.2081, 0.9298, 0.7231, 0.7423],
        [0.5263, 0.2437, 0.5846, 0.0332, 0.1387],
        [0.2422, 0.8155, 0.7932, 0.2783, 0.4820],
        [0.8198, 0.9971, 0.6984, 0.5675, 0.8352],
        [0.2056, 0.5932, 0.1123, 0.1535, 0.2417],
        [0.7262, 0.7011, 0.2038, 0.6511, 0.7745]], requires_grad=True) 

tensor([[0.4369, 0.5191, 0.6159],
        [0.8102, 0.9801, 0.1147],
        [0.3168, 0.6965, 0.9143],
        [0.9351, 0.9412, 0.5995],
        [0.0652, 0.5460, 0.1872]], requires_grad=True) 

tensor([[0.0340],
        [0.9442],
        [0.8802]], requires_grad=True) 



In [ ]:
global weights



def training_step(step, lr):
    global weights
    
    loss = calc_loss(trn_indep, weights, trn_dep)
    loss.backward()
    
    with torch.no_grad():
        weights.sub_(weights.grad * lr)
        weights.grad.zero_()
        
        if step % 1000 == 0:
            trn_loss = calc_loss(trn_indep, weights, trn_dep).item()
            val_loss = calc_loss(val_indep, weights, val_dep).item()
            print(trn_loss, val_loss)
        
        
def train(steps=10000, lr=0.1):
    torch.manual_seed(0)
    random.seed(0)
    np.random.seed(0)
    
    global weights
    weights = torch.rand(train_df.shape[1]) - 0.5
    weights.requires_grad_()
    
    for i in range(steps):
        training_step(i, lr)